In [3]:
import cv2
import serial
import time
from collections import defaultdict
from ultralytics import YOLO

# =========================
# 1. 모델 설정
# =========================
MODEL_PATH = r"C:/ESG_Project2/yolov12n_train_result/weights/best.pt"
model = YOLO(MODEL_PATH)

# =========================
# 2. Arduino 연결
# =========================
ser = serial.Serial("COM8", 9600, timeout=1)
time.sleep(2)
print("아두이노 연결 성공!")

# =========================
# 3. 테스트 이미지 경로
# =========================
IMAGE_PATH = r"C:\Users\배명훈\Downloads\손상.webp"   # ← 테스트할 이미지 경로

# =========================
# 4. 색상 정의 (BGR)
# =========================
COLOR_SEARCH  = (80, 80, 80)     # 진한 회색
COLOR_NORMAL  = (255, 0, 0)      # 파란색
COLOR_WARNING = (0, 0, 255)      # 빨간색
COLOR_TOTAL   = (60, 20, 220)    # 크림슨

CLASS_COLORS = {
    "Normal Solar Panel": (255, 0, 0),        # 파란색
    "Snow": (255, 255, 0),                    # 하늘색
    "Structural Damage": (128, 0, 128),       # 보라색
    "Surface Contaminants": (0, 165, 255),    # 주황색
    "Anomaly Solar Panel": (255, 0, 255)      # 진한 분홍
}

# =========================
# 5. 이미지 탐지 함수
# =========================
def detect_image_and_alert(image_path):
    frame = cv2.imread(image_path)
    if frame is None:
        print("이미지를 찾을 수 없습니다.")
        return

    results = model(frame, conf=0.25, verbose=False)
    r = results[0]

    status_msg = "SEARCHING..."
    status_color = COLOR_SEARCH
    signal = b'O'

    total_count = 0
    class_counts = defaultdict(int)
    anomaly_detected = False

    # =========================
    # 탐지 결과 분석
    # =========================
    if r.boxes is not None and len(r.boxes) > 0:
        total_count = len(r.boxes)

        status_msg = "NORMAL"
        status_color = COLOR_NORMAL
        signal = b'N'

        for box in r.boxes:
            class_id = int(box.cls[0])
            class_name = model.names[class_id]
            class_counts[class_name] += 1

            if class_name != "Normal Solar Panel":
                anomaly_detected = True
                status_msg = "WARNING"
                status_color = COLOR_WARNING
                signal = b'A'

    # =========================
    # Arduino 제어 (부저 3초)
    # =========================
    if anomaly_detected:
        ser.write(b'A')
        time.sleep(3)       # 🔊 3초간 부저 울림
        ser.write(b'O')
    else:
        ser.write(b'N')

    # =========================
    # 결과 화면 표시
    # =========================
    annotated = r.plot()

    cv2.putText(
        annotated,
        status_msg,
        (20, 35),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        status_color,
        3
    )

    cv2.putText(
        annotated,
        f"TOTAL: {total_count}",
        (20, 75),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.1,
        COLOR_TOTAL,
        4
    )

    y_offset = 115
    for cls_name, count in class_counts.items():
        color = CLASS_COLORS.get(cls_name, (255, 255, 255))
        cv2.putText(
            annotated,
            f"{cls_name}: {count}",
            (20, y_offset),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.75,
            color,
            2
        )
        y_offset += 30

    cv2.imshow("Solar Panel Image Detection", annotated)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# =========================
# 실행
# =========================
detect_image_and_alert(IMAGE_PATH)

# =========================
# 종료 처리
# =========================
ser.write(b'O')
ser.close()


아두이노 연결 성공!


In [1]:
import cv2
import serial
import time
from collections import defaultdict
from ultralytics import YOLO

# =========================
# 1. 모델 설정
# =========================
MODEL_PATH = r"C:/ESG_Project2/yolov12n_train_result/weights/best.pt"
model = YOLO(MODEL_PATH)

# =========================
# 2. Arduino 연결
# =========================
ser = serial.Serial("COM8", 9600, timeout=1)
time.sleep(2)
print("아두이노 연결 성공!")

# =========================
# 3. 웹캠 실행
# =========================
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

last_signal = b'O'  # 이전 상태 저장

# =========================
# 4. 색상 정의 (BGR)
# =========================
COLOR_SEARCH  = (80, 80, 80)     # 진한 회색
COLOR_NORMAL  = (255, 0, 0)      # 파란색
COLOR_WARNING = (0, 0, 255)      # 빨간색
COLOR_TOTAL   = (60, 20, 220)    # 크림슨

CLASS_COLORS = {
    "Normal Solar Panel": (0, 255, 0),        # 초록색
    "Snow": (255, 255, 0),                    # 하늘색
    "Structural Damage": (128, 0, 128),       # 보라색
    "Surface Contaminants": (0, 165, 255),    # 주황색
    "Anomaly Solar Panel": (255, 0, 255)      # 진한 분홍
}

try:
    while True:
        success, frame = cap.read()
        if not success:
            continue

        # YOLO 추론
        results = model(frame, conf=0.25, verbose=False)
        r = results[0]

        # =========================
        # 기본 상태 (SEARCHING)
        # =========================
        signal = b'O'
        status_msg = "SEARCHING..."
        status_color = COLOR_SEARCH

        total_count = 0
        class_counts = defaultdict(int)

        # =========================
        # 탐지 결과 분석
        # =========================
        if r.boxes is not None and len(r.boxes) > 0:
            total_count = len(r.boxes)

            signal = b'N'
            status_msg = "NORMAL"
            status_color = COLOR_NORMAL

            for box in r.boxes:
                class_id = int(box.cls[0])
                class_name = model.names[class_id]
                class_counts[class_name] += 1

                # Normal이 아닌 것이 하나라도 있으면 WARNING
                if class_name != "Normal Solar Panel":
                    signal = b'A'
                    status_msg = "WARNING"
                    status_color = COLOR_WARNING

        # =========================
        # 상태 변경 시만 Arduino 전송
        # =========================
        if signal != last_signal:
            ser.write(signal)
            last_signal = signal

        # =========================
        # 화면 출력
        # =========================
        annotated = r.plot()

        # 상태 표시
        cv2.putText(
            annotated,
            status_msg,
            (20, 35),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            status_color,
            3
        )

        # TOTAL 표시
        cv2.putText(
            annotated,
            f"TOTAL: {total_count}",
            (20, 75),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,              
            COLOR_TOTAL,
            4                 
        )

        # 클래스별 개수 표시
        y_offset = 105
        for cls_name, count in class_counts.items():
            color = CLASS_COLORS.get(cls_name, (255, 255, 255))

            cv2.putText(
                annotated,
                f"{cls_name}: {count}",
                (20, y_offset),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.75,
                color,
                2
            )
            y_offset += 30

        cv2.imshow("Solar Panel Monitor", annotated)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    if ser.is_open:
        ser.write(b'O')
        ser.close()

    cap.release()
    cv2.destroyAllWindows()


아두이노 연결 성공!
